# Teste Técnico para Analista de Dados (BUS2)

In [52]:
import pandas as pd
import os
import re
import pyarrow
import streamlit as st
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

## Passo 1: Importação dos dados

In [53]:
DATA_PATH = "data"

empregados_path = os.path.join(DATA_PATH, "empregados.csv")
produtos_path = os.path.join(DATA_PATH, "produtos.csv")
vendas_path = os.path.join(DATA_PATH, "vendas.csv")



df_empregados = pd.read_csv("data/empregados.csv")
df_produtos = pd.read_csv("data/produtos.csv")
df_vendas = pd.read_csv("data/vendas.csv")


### Leitura dos dados

In [54]:
print("Empregados")
display(df_empregados.head())

print("Produtos")
display(df_produtos.head())

print("Vendas")
display(df_vendas.head())


Empregados


,id_empregado,nome,cargo,idade
0,1,Júlia Oliveira,Vendedor,NaN
1,2,Diogo Vieira,NaN,49.0
2,3,Sr. Rodrigo das Neves,Assistente,56.0
3,4,Isis Castro,Gerente,49.0
4,5,Leandro Azevedo,Vendedor,21.0


Produtos


,id_produto,nome,preco,categoria
0,1,Produto 1,988.59,Livros
1,2,Produto 2,171.15,Roupas
2,3,Produto 3,187.95,Roupas
3,4,Produto 4,1206.87,Casa
4,5,Produto 5,1109.64,Roupas


Vendas


,id_venda,data,id_produto,id_empregado,quantidade,valor_unitario,valor_total
0,1,2023-03-16,70,18,2,279.58,559.16
1,2,2023-02-10,166,34,5,816.90,4084.50
2,3,2023-05-02,76,87,1,411.20,411.20
3,4,2023-01-27,57,31,5,20.89,104.45
4,5,2023-04-08,12,53,6,734.86,4409.16


## Passo 2: Transformação dos Dados

### Removendo duplicatas

In [55]:
df_empregados = df_empregados.drop_duplicates()
df_produtos = df_produtos.drop_duplicates()
df_vendas = df_vendas.drop_duplicates()



### Verificando valores Ausentes

In [ ]:
print("Valores ausentes - Empregados:")
print(df_empregados.isnull().sum())

print("Valores ausentes - Produtos:")
print(df_produtos.isnull().sum())

print("Valores ausentes - Vendas:")
print(df_vendas.isnull().sum())




In [ ]:
df["preco"] = df.groupby("categoria")["preco"].transform(lambda x: x.fillna(x.mean()))

In [89]:
# substituição de dados ausentes com média
df_empregados['idade'] = df_empregados['idade'].fillna(df_empregados['idade'].mean())

# Cargo ausente = “DESCONHECIDO”
df_empregados['cargo'] = df_empregados['cargo'].fillna('DESCONHECIDO')

# Nome ausente = “NÃO INFORMADO”
df_empregados['nome'] = df_empregados['nome'].fillna('NÃO INFORMADO')

df_empregados["idade"] = df_empregados["idade"].round(0).astype(int)
df_vendas["preco"] = df.groupby("categoria")["preco"].transform(lambda x: x.fillna(x.mean()))

# Substituiindo valores NULL da tabela preço de acordo comos valores por categoria 
df_vendas["preco"] = df["preco"].fillna(df.groupby("categoria")["preco"].transform("mean"))


### Retirando Siglas dos nomes da tabela Empregados

In [91]:

df_empregados['nome'] = df_empregados['nome'].apply(lambda x: re.sub(r'\b(Sr\.|Sra\.|Dr\.|Dra\.)\s*', '', x))

# Uniformizar texto
df_empregados['nome'] = df_empregados['nome'].str.strip().str.title()
df_empregados['cargo'] = df_empregados['cargo'].str.strip().str.upper()


### Checagem de integridade 

In [92]:

vendas_ids_validos = df_vendas['id_empregado'].isin(df_empregados['id_empregado'])
print(f"IDs inválidos de empregados em vendas: {df_vendas[~vendas_ids_validos].shape[0]}")

produtos_ids_validos = df_vendas['id_produto'].isin(df_produtos['id_produto'])
print(f"IDs inválidos de produtos em vendas: {df_vendas[~produtos_ids_validos].shape[0]}")

IDs inválidos de empregados em vendas: 0
IDs inválidos de produtos em vendas: 0


In [ ]:
# Excluir linhas incompletas em vendas
df = df_vendas.dropna(subset=['quantidade', 'valor_unitario'], inplace=True)

    # Substituir nome ou categoria ausente 
    df_produtos['nome'] = df_produtos['nome'].fillna('PRODUTO NÃO INFORMADO')
    df_produtos['categoria'] = df_produtos['categoria'].fillna('DESCONHECIDA')

    df_vendas["preco"] = pd.to_numeric(df["preco"], errors="coerce")
    df_vendas["preco"] = df["preco"].fillna(df.groupby("categoria")["preco"].transform("mean"))


### SAlvando os dados tratados para serem usados no Power BI para melhor visualização.

In [94]:
df_empregados.to_parquet("dados_tratados/empregados_tratado.parquet", engine="pyarrow", index=False)
df_produtos.to_parquet("dados_tratados/produtos_tratado.parquet", engine="pyarrow", index=False)
df_vendas.to_parquet("dados_tratados/vendas_tratado.parquet", engine="pyarrow", index=False)



## Juntando as tabelas com Merge 

In [102]:

df_merged = df_vendas.merge(df_empregados, on='id_empregado', how='left')
df_merged = df_merged.merge(df_produtos, on='id_produto', how='left')

df_merged['valor_total'] = df_merged['quantidade'] * df_merged['valor_unitario']

df_resumo = df_merged.copy()


df_resumo.to_csv("dados_tratados/resumo-vendas.csv", index=False, encoding="utf-8-sig")
df_resumo.to_parquet("dados_tratados/resumo-vendas.parquet", engine="pyarrow", index=False)     

In [98]:
print(df_resumo[df_resumo["data"].isnull()])


     id_venda data  id_produto  id_empregado  quantidade  valor_unitario  \
9          10  NaN          54            98           7          614.30   
18         20  NaN         149            81           3          734.90   
27         30  NaN          93            86           8          544.10   
36         40  NaN         140            59           9          682.85   
46         50  NaN         101            98           9          895.97   
..        ...  ...         ...           ...         ...             ...   
886       960  NaN         134            69           9           68.51   
895       970  NaN         118            45          10          875.56   
904       980  NaN         151            96           7          488.96   
913       990  NaN         192            88           8           40.62   
923      1000  NaN          99            52           1           29.94   

     valor_total  preco_x               nome_x       cargo  idade  \
9        4300.10  

## Calculando os dados Pedidos como: 
>- Total de vendas por funcionário
>- Ticket médio por produto
>- Vendas por categoria
>- Top 5 vendedores



In [ ]:


# Configurações visuais
sns.set(style="whitegrid")
pdf_path = "relatorio-final.pdf"

# KPIs
receita_total = df_resumo["valor_total"].sum()
num_vendas = df_resumo["id_venda"].nunique()
ticket_medio = receita_total / max(num_vendas, 1)

# Tabelas
total_func = (
    df_resumo.groupby("nome_funcionario")
    .agg(receita_total=("valor_total", "sum"),
         num_vendas=("id_venda", "nunique"))
    .reset_index()
    .sort_values("receita_total", ascending=False)
)

ticket_prod = (
    df_resumo.groupby("nome_produto")
    .agg(receita_total=("valor_total", "sum"),
         num_vendas=("id_venda", "nunique"))
    .reset_index()
)
ticket_prod["ticket_medio"] = ticket_prod["receita_total"] / ticket_prod["num_vendas"]
ticket_prod = ticket_prod.sort_values("ticket_medio", ascending=False)

cat = df_resumo.groupby("categoria")["valor_total"].sum().reset_index()
cat = cat.sort_values("valor_total", ascending=False)

# Geração do PDF
with PdfPages(pdf_path) as pdf:
    # Página 1 – KPIs
    fig, ax = plt.subplots(figsize=(11.69, 8.27))  # A4 landscape
    ax.axis("off")
    ax.text(0.05, 0.95, "📊 Relatório Final – BUS2", fontsize=20, weight="bold")
    ax.text(0.05, 0.9, f"Receita total: R$ {receita_total:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))
    ax.text(0.05, 0.86, f"Nº de vendas: {num_vendas:,}".replace(",", "."))
    ax.text(0.05, 0.82, f"Ticket médio: R$ {ticket_medio:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))

    top5 = total_func.nlargest(5, "receita_total").copy()
    top5["receita_total"] = top5["receita_total"].map(lambda x: f"R$ {x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))
    tbl = ax.table(cellText=top5.values,
                   colLabels=top5.columns,
                   loc="center right",
                   colLoc="left",
                   cellLoc="left")
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(8)
    tbl.scale(1, 1.5)

    pdf.savefig(fig)
    plt.close(fig)

    # Página 2 – Receita por funcionário
    fig, ax = plt.subplots(figsize=(11.69, 8.27))
    top5 = total_func.nlargest(5, "receita_total")
    sns.barplot(data=top5, y="nome_funcionario", x="receita_total", ax=ax)
    ax.set_title("Receita por Funcionário (Top 5)")
    ax.set_xlabel("Receita (R$)")
    ax.set_ylabel("")
    pdf.savefig(fig)
    plt.close(fig)

    # Página 3 – Receita por categoria
    fig, ax = plt.subplots(figsize=(11.69, 8.27))
    sns.barplot(data=cat, y="categoria", x="valor_total", ax=ax)
    ax.set_title("Receita por Categoria")
    ax.set_xlabel("Receita (R$)")
    ax.set_ylabel("")
    pdf.savefig(fig)
    plt.close(fig)

    # Página 4 – Ticket médio por produto
    fig, ax = plt.subplots(figsize=(11.69, 8.27))
    top_tp = ticket_prod.nlargest(10, "ticket_medio")
    sns.barplot(data=top_tp, y="nome_produto", x="ticket_medio", ax=ax)
    ax.set_title("Ticket Médio por Produto (Top 10)")
    ax.set_xlabel("Ticket médio (R$)")
    ax.set_ylabel("")
    pdf.savefig(fig)
    plt.close(fig)

    # Página 5 – Metadados
    fig, ax = plt.subplots(figsize=(11.69, 8.27))
    ax.axis("off")
    ax.text(0.05, 0.95, "🧾 Metadados do resumo de vendas", fontsize=16, weight="bold")
    ax.text(0.05, 0.9, f"Nº de linhas: {len(df_resumo):,}".replace(",", "."))
    ax.text(0.05, 0.86, f"Nº de colunas: {len(df_resumo.columns)}")
    ax.text(0.05, 0.82, f"Colunas: {', '.join(df_resumo.columns)}")
    pdf.savefig(fig)
    plt.close(fig)

print("✅ Relatório gerado com sucesso: relatorio-final.pdf")

In [99]:
conn = sqlite3.connect("resumo-vendas.db")
df_resumo.to_sql("resumo_vendas", conn, if_exists="replace", index=False)
conn.close()